In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/24 19:32:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/24 19:32:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/02/24 19:32:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/home/lilychau/spark/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [21]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    count(1) as number_records
FROM
    green
where
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
order by
    1, 2
""")

In [24]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [25]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    count(1) as number_records
FROM
    yellow
where
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
order by
    1, 2
""")

/home/lilychau/spark/spark-3.4.2-bin-hadoop3/python/pyspark/sql/dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [26]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [36]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [37]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [38]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [39]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [40]:
df_join = spark.read.parquet('data/report/revenue/total')

In [42]:
df_zones = spark.read.parquet('zones/')

In [46]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [49]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')